In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from matplotlib import pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
from torch.utils.data import random_split

dataset = datasets.OxfordIIITPet(
    root='./data',
    split='trainval',
    target_types="category",
    transform=transform,
    download=True
)

test_dataset = datasets.OxfordIIITPet(
    root='./data',
    split='test',
    target_types="category",
    transform=transform,
    download=True
)

classes = dataset.classes
num_classes = len(dataset.classes)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
def create_model(num_classes):
    model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier[6] = nn.Linear(4096, num_classes)
    return model.to(device)

In [ ]:
def run_epoch(model, loader, criterion, optimizer=None):
    training = optimizer is not None
    model.train() if training else model.eval()

    total_loss, correct, total = 0, 0, 0

    context = torch.enable_grad() if training else torch.no_grad()
    with context:
        for x, y in tqdm(loader, leave=False):
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = criterion(output, y)

            if training:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            preds = output.argmax(dim=1)
            correct += preds.eq(y).sum().item()
            total += y.size(0)

    return total_loss / len(loader), correct / total

In [ ]:
def train_model(model, epochs, lr):
    optimizer = optim.Adam(model.classifier.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    for epoch in range(epochs):
        train_loss, train_acc = run_epoch(
            model, train_loader, criterion, optimizer
        )

        val_loss, val_acc = run_epoch(
            model, val_loader, criterion
        )

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"TrainLoss: {train_loss:.4f} | "
            f"ValLoss: {val_loss:.4f} | "
            f"TrainAcc: {train_acc*100:.2f}% | "
            f"ValAcc: {val_acc*100:.2f}%"
        )

    return history

In [ ]:
def plot_training_curves(results):

    plt.figure(figsize=(12, 6))

    plt.subplot(2,2,1)
    for name, history in results.items():
        plt.plot(history["train_loss"], label=f"{name} Train Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.legend()

    plt.subplot(2,2,2)
    for name, history in results.items():
        plt.plot(history["val_loss"], label=f"{name} Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Validation Loss")
    plt.legend()

    plt.subplot(2,2,3)
    for name, history in results.items():
        plt.plot(history["train_acc"], label=f"{name} Train Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy")
    plt.legend()

    plt.subplot(2,2,4)
    for name, history in results.items():
        plt.plot(history["val_acc"], label=f"{name} Val Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Validation Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
results = {}

for epoch in [5, 10, 15]:
    print(f"\nTraining with epochs={epoch}")
    model = create_model(num_classes)
    history = train_model(model, epochs=epoch, lr=0.001)
    results[epoch] = history

plot_training_curves(results)

In [ ]:
lrs = [1e-2, 1e-3, 1e-4]
results = {}

for lr in lrs:
    print(f"\nTraining with LR={lr}")
    model = create_model(num_classes)
    history = train_model(model, epochs=5, lr=lr)
    results[f"lr={lr}"] = history

plot_training_curves(results)

In [ ]:
def show_predictions(model, loader, device, classes, n=8):
    model.eval()

    images, labels = next(iter(loader))
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

    images = images.cpu()
    labels = labels.cpu()
    predicted = predicted.cpu()

    plt.figure(figsize=(12, 3))
    for i in range(n):
        plt.subplot(1, n, i+1)
        img = images[i].permute(1, 2, 0)
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        img = torch.clamp(img, 0, 1)

        plt.imshow(img)
        plt.title(f"P: {classes[predicted[i]]}\nT: {classes[labels[i]]}", fontsize=8)
        plt.axis('off')
    plt.show()

In [ ]:
best_model = create_model(num_classes)
history = train_model(best_model, epochs=5, lr=0.001)
show_predictions(best_model, test_loader, device, classes, n=8)